In [ ]:
# Following https://radimrehurek.com/topic_modeling_tutorial/2%20-%20Topic%20Modeling.html

In [6]:
import gensim
import glob
import itertools
import os

In [38]:
from nltk.tokenize import word_tokenize
from gensim.parsing.preprocessing import STOPWORDS

In [3]:
def head(stream, n=10):
    return list(itertools.islice(stream, n))

In [13]:
# Given a filename, extract just the book id
def get_book_id(fn):
    return os.path.split(fn)[1].split('_')[0].split('.')[0]

In [39]:
def tokenize(text):
    return [token for token in word_tokenize(text) if token not in STOPWORDS]

In [40]:
def text_stream(text_dir):
    #     yield each article as (title, tokens) tuple
    for fn in glob.glob('tm_texts/*.txt'):
        book_id = get_book_id(fn)
        with open(fn, 'r') as f:
            document = f.read()
            yield (book_id, tokenize(document))

In [41]:
text_dir = "tm_texts"

In [42]:
head(text_stream(text_dir))

[('36105213320042',
  ['florence',
   'montgomerythwartedlondon',
   'printed',
   'ily',
   'spottiswoodb',
   'new',
   'square',
   'parliament',
   'streetthwarted',
   'ducks',
   'eggs',
   'hen',
   'nest',
   'hy',
   'author',
   'misunderstood',
   'london',
   'richard',
   'bentley',
   'son',
   'new',
   'burlington',
   'street',
   'rights',
   'reservesito',
   'youngest',
   'brother',
   'alec',
   'cjjis',
   'iltlc',
   'slorg',
   'jlíbictütbcontents',
   'chapter',
   'page',
   'washerwoman',
   'ner',
   'family',
   'ii',
   'quiet',
   'little',
   'village',
   'grinfield',
   'iii',
   'brother',
   'sister',
   'walk',
   'iiome',
   'iv',
   'trying',
   'past',
   'fairy',
   'child',
   'big',
   'house',
   'vi',
   'letter',
   'bourn',
   'ii',
   'vii',
   'burden',
   'double',
   'secret',
   'viii',
   'parable',
   'talents',
   'vi',
   'contexts',
   'chattei',
   'paoe',
   'ix',
   'sister',
   'fairy',
   'child',
   'hie',
   'infant',
   

In [43]:
for book_id, tokens in itertools.islice(text_stream(text_dir), 5):
    print(book_id, tokens[:10])

36105213320042 ['florence', 'montgomerythwartedlondon', 'printed', 'ily', 'spottiswoodb', 'new', 'square', 'parliament', 'streetthwarted', 'ducks']
36105213325884 ['ÿê', 'êêê', 'rrr', 'jk', 'ws', 'êàthe', 'secret', 'mere', 'stir', 'face']
36105213324556 ['blind', 'larry', 'irish', 'idylls', 'lewis', 'macnamara', 'blind', 'larry', 'impressionist', 'series']
36105213325660 ['si', 'ules', 'samemorrison', 'gibb', 'irnters', 'edinburghhe', 'si', 'ules', 'ame', 'roger']
36105213334779 ['doctor', 'ditr', 'inrinry', 'author', 'tales', 'kirkbeck', 'alice', 'beresford', 'lives', 'fathers']


In [44]:
doc_stream = (tokens for _, tokens in text_stream(text_dir))

In [45]:
%time id2word_novels = gensim.corpora.Dictionary(doc_stream)

CPU times: user 6min 30s, sys: 3.93 s, total: 6min 34s
Wall time: 6min 38s


In [21]:
print(id2word_novels)

Dictionary(417787 unique tokens: ['aa', 'abideth', 'able', 'ably', 'about']...)


In [23]:
class NovelCorpus(object):
    def __init__(self, text_dir, dictionary):
        self.text_dir = text_dir
        self.dictionary = dictionary
        
    def __iter__(self):
        self.book_ids = []
        for book_id, tokens in text_stream(text_dir):
            self.book_ids.append(book_id)
            yield self.dictionary.doc2bow(tokens)

In [46]:
novel_corpus = NovelCorpus(text_dir, id2word_novels)

In [25]:
vector = next(iter(novel_corpus))
print(vector)

[(0, 1), (1, 1), (2, 15), (3, 2), (4, 67), (5, 1), (6, 1), (7, 1), (8, 1), (9, 2), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 2), (20, 1), (21, 2), (22, 12), (23, 1), (24, 1), (25, 1), (26, 5), (27, 1), (28, 5), (29, 2), (30, 1), (31, 1), (32, 2), (33, 1), (34, 1), (35, 1), (36, 13), (37, 2), (38, 2), (39, 1), (40, 1), (41, 1), (42, 1), (43, 1), (44, 1), (45, 1), (46, 3), (47, 1), (48, 2), (49, 5), (50, 1), (51, 8), (52, 1), (53, 1), (54, 1), (55, 1), (56, 4), (57, 1), (58, 1), (59, 13), (60, 37), (61, 9), (62, 2), (63, 35), (64, 5), (65, 1), (66, 3), (67, 1), (68, 5), (69, 5), (70, 1), (71, 1), (72, 7), (73, 2), (74, 1), (75, 3), (76, 8), (77, 1), (78, 1), (79, 1), (80, 1), (81, 1), (82, 1), (83, 3), (84, 1), (85, 1), (86, 3), (87, 140), (88, 1), (89, 1), (90, 1), (91, 3), (92, 1), (93, 2), (94, 14), (95, 9), (96, 12), (97, 1), (98, 5), (99, 5), (100, 10), (101, 19), (102, 16), (103, 1), (104, 3), (105, 3), (106, 1), (107, 1), (108, 1), (109,

In [47]:
%time gensim.corpora.MmCorpus.serialize("./novels_bow.mm", novel_corpus)

CPU times: user 6min 30s, sys: 3.72 s, total: 6min 34s
Wall time: 6min 37s


In [48]:
j_corpus = gensim.corpora.MmCorpus("./novels_bow.mm")

In [49]:
print(j_corpus)

MmCorpus(1417 documents, 417455 features, 10042903 non-zero entries)


In [51]:
%time lda_model = gensim.models.LdaModel(j_corpus, num_topics=40, id2word=id2word_novels, passes=10)

CPU times: user 43min 28s, sys: 7min 26s, total: 50min 54s
Wall time: 13min 54s


In [58]:
lda_model.save('novels_50.model')

In [53]:
lda_model.print_topics(-1)

[(0,
  '0.006*"said" + 0.004*"man" + 0.004*"great" + 0.004*"men" + 0.004*"time" + 0.004*"grace" + 0.003*"king" + 0.003*"thou" + 0.003*"shall" + 0.003*"life"'),
 (1,
  '0.006*"said" + 0.004*"time" + 0.004*"man" + 0.003*"men" + 0.003*"little" + 0.003*"like" + 0.003*"old" + 0.003*"great" + 0.003*"good" + 0.002*"way"'),
 (2,
  '0.020*"ye" + 0.006*"wi" + 0.006*"said" + 0.005*"ll" + 0.005*"man" + 0.004*"time" + 0.004*"like" + 0.004*"hae" + 0.004*"laird" + 0.003*"weel"'),
 (3,
  '0.003*"time" + 0.003*"said" + 0.003*"jim" + 0.003*"father" + 0.003*"god" + 0.003*"man" + 0.002*"great" + 0.002*"heart" + 0.002*"soon" + 0.002*"day"'),
 (4,
  '0.008*"said" + 0.005*"clara" + 0.004*"little" + 0.004*"lady" + 0.004*"italy" + 0.004*"time" + 0.003*"know" + 0.003*"exiles" + 0.003*"farnorth" + 0.003*"man"'),
 (5,
  '0.011*"vo" + 0.006*"said" + 0.006*"crown" + 0.006*"cloth" + 0.005*"edition" + 0.005*"man" + 0.004*"little" + 0.003*"life" + 0.003*"old" + 0.003*"time"'),
 (6,
  '0.012*"said" + 0.005*"man" + 0.00

In [59]:
import pyLDAvis
import pyLDAvis.gensim

In [60]:
pyLDAvis.enable_notebook()

In [62]:
pyLDAvis.gensim.prepare(lda_model, j_corpus, id2word_novels)

/Users/csb5t/.local/share/virtualenvs/barth-R-kcokRtVH/lib/python3.6/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
16     0.020637 -0.064712       1        1  10.007313
14     0.006397 -0.021240       2        1   9.006236
33    -0.023320  0.007982       3        1   7.000969
5      0.009059 -0.045044       4        1   5.857079
18     0.011885 -0.034719       5        1   5.286286
34     0.004892 -0.012848       6        1   4.834909
8     -0.045330  0.001063       7        1   4.753710
1     -0.030610 -0.012476       8        1   4.668166
25    -0.048452  0.012583       9        1   4.031826
24    -0.060191  0.035665      10        1   3.772727
11    -0.008088 -0.014201      11        1   3.579527
17    -0.059896  0.026166      12        1   3.534160
38     0.001178 -0.021430      13        1   3.346743
32    -0.014633  0.003276      14        1   3.226779
19    -0.018823 -0.011149      15        1   2.869513
15    -0.024057  0.007462      16        1   2.568448
0     -0.082486  0.038198      17        1   2.255942
21    -0.027123 -0.006886      18        1   2.034094
20     0.000994 -0.048860      19        1   1.972195
35    -0.000845 -0.019162      20        1   1.942538
30     0.004621 -0.022723      21        1   1.914951
2     -0.002300 -0.053322      22        1   1.766773
22    -0.001713 -0.055690      23        1   1.651934
37     0.007157 -0.037808      24        1   1.126335
6     -0.000761 -0.014480      25        1   0.961847
3     -0.077948  0.085696      26        1   0.869888
29     0.020746 -0.016645      27        1   0.614258
27     0.006235 -0.003214      28        1   0.598868
36     0.039114 -0.045982      29        1   0.531825
12     0.017712  0.014302      30        1   0.416065
9      0.030480 -0.019243      31        1   0.407432
7      0.008634  0.032845      32        1   0.399624
28     0.023996  0.015268      33        1   0.396590
13    -0.017558  0.062126      34        1   0.388048
10     0.019105  0.021756      35        1   0.351186
31     0.041362 -0.000033      36        1   0.302709
4      0.015602  0.038626      37        1   0.292974
23     0.049769  0.022682      38        1   0.218190
26     0.027008  0.070774      39        1   0.195977
39     0.177550  0.085398      40        1   0.045374, topic_info=       Category           Freq     Term          Total  loglift  logprob
term                                                                    
2425    Default  374041.000000     said  374041.000000  30.0000  30.0000
8627    Default   49904.000000       vo   49904.000000  29.0000  29.0000
1703    Default  182749.000000   little  182749.000000  28.0000  28.0000
2907    Default  167088.000000     time  167088.000000  27.0000  27.0000
1755    Default  183244.000000      man  183244.000000  26.0000  26.0000
1968    Default  138372.000000      old  138372.000000  25.0000  25.0000
1591    Default  137717.000000     know  137717.000000  24.0000  24.0000
1680    Default  156472.000000     like  156472.000000  23.0000  23.0000
1242    Default  133047.000000     good  133047.000000  22.0000  22.0000
1830    Default   78438.000000     miss   78438.000000  21.0000  21.0000
501     Default  120882.000000     come  120882.000000  20.0000  20.0000
2875    Default  114979.000000    think  114979.000000  19.0000  19.0000
3257    Default   37876.000000       ye   37876.000000  18.0000  18.0000
1038    Default   93407.000000   father   93407.000000  17.0000  17.0000
653     Default  110042.000000      day  110042.000000  16.0000  16.0000
486     Default   37068.000000    cloth   37068.000000  15.0000  15.0000
1602    Default   87664.000000     lady   87664.000000  14.0000  14.0000
1674    Default  113825.000000     life  113825.000000  13.0000  13.0000
3152    Default  100601.000000      way  100601.000000  12.0000  12.0000
362     Default   98251.000000     came   98251.000000  11.0000  11.0000
7722    Default   80605.000000      sir   80605.000000  10.0000  10.0000
2880    De

In [ ]:
# Need to clean word 'said'
# Probably need to remove proper names
# remove words like 'thy,' 'thou,' etc.